<a href="https://colab.research.google.com/github/JuanAntCB/Caffeine-nb-mod-for-AFL/blob/main/nb_caffeine_mod_for_AFL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyrosettacolabsetup
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()
import pyrosetta; pyrosetta.init()
from pyrosetta import *
import sys
import matplotlib
init()

Mounted at /content/google_drive

Note that USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE.
See https://github.com/RosettaCommons/rosetta/blob/main/LICENSE.md or email license@uw.edu for details.

Looking for compatible PyRosetta wheel file at google-drive/PyRosetta/colab.bin//wheels...
Found compatible wheel: /content/google_drive/MyDrive/PyRosetta/colab.bin/wheels//content/google_drive/MyDrive/PyRosetta/colab.bin/wheels/pyrosetta-2024.1+release.00b79147e63-cp310-cp310-linux_x86_64.whl


PyRosetta-4 2023 [Rosetta PyRosetta4.MinSizeRel.python310.ubuntu 2024.01+release.00b79147e63be743438188f93a3f069ca75106d6 2023-12-25T16:35:48] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python310.ubuntu r366 2024.01+release.00b79147e63 00b79147e63be743438188f93a3f06

In [2]:
from google.colab import drive
import os

drive.flush_and_unmount()

drive.mount('/content/drive')

path = '/content/drive/MyDrive/nb_caffeine_mod_for_AFL/Inputs/aftNb_start.pdb'
path_aft = '/content/drive/MyDrive/nb_caffeine_mod_for_AFL/Inputs/aft.sdf'
path_params = '/content/drive/MyDrive/nb_caffeine_mod_for_AFL/Inputs/AFT.params'
drive_path = '/content/drive/MyDrive/nb_caffeine_mod_for_AFL/Inputs'


Mounted at /content/drive


In [3]:
pose = pose_from_pdb(path)



core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 1.4527 seconds.
core.import_pose.import_pose: File '/content/drive/MyDrive/nb_caffeine_mod_for_AFL/Inputs/aftNb_start.pdb' automatically determined to be of type PDB
core.chemical.GlobalResidueTypeSet: Loading (but possibly not actually using) 'AFT' from the PDB components dictionary for residue type 'pdb_AFT'
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 12 atoms at position 121 in file /content/drive/MyDrive/nb_caffeine_mod_for_AFL/Inputs/aftNb_start.pdb. Best match rsd_type:  pdb_AFT
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue VAL:CtermProteinFull 238


In [4]:
ligand_parameters = path_params
flags = f"""
-extra_res_fa {ligand_parameters}
-s path
-ex1
-ex2
-no_optH false
-ignore_ligand_chi true
-overwrite
-restore_pre_talaris_2013_behavior true
"""
pyrosetta.distributed.init(flags)

In [5]:
pose = pyrosetta.io.pose_from_file(filename = path)

core.import_pose.import_pose: File '/content/drive/MyDrive/nb_caffeine_mod_for_AFL/Inputs/aftNb_start.pdb' automatically determined to be of type PDB
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 12 atoms at position 121 in file /content/drive/MyDrive/nb_caffeine_mod_for_AFL/Inputs/aftNb_start.pdb. Best match rsd_type:  pdb_AFT
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue VAL:CtermProteinFull 238


In [28]:
xml = pyrosetta.rosetta.protocols.rosetta_scripts.XmlObjects.create_from_string("""
<ROSETTASCRIPTS>

		<SCOREFXNS>
			<ScoreFunction name="ligand_soft_rep" weights="ligand_soft_rep">
			</ScoreFunction>
			<ScoreFunction name="hard_rep" weights="ligand">
			</ScoreFunction>
		</SCOREFXNS>

		<LIGAND_AREAS>
			<LigandArea name="inhibitor_dock_sc" chain="A" cutoff="6.0" add_nbr_radius="true" all_atom_mode="false"/>
			<LigandArea name="inhibitor_final_sc" chain="A" cutoff="6.0" add_nbr_radius="true" all_atom_mode="false"/>
			<LigandArea name="inhibitor_final_bb" chain="A" cutoff="7.0" add_nbr_radius="false" all_atom_mode="true" Calpha_restraints="0.3"/>
		</LIGAND_AREAS>

		<INTERFACE_BUILDERS>
			<InterfaceBuilder name="side_chain_for_docking" ligand_areas="inhibitor_dock_sc"/>
			<InterfaceBuilder name="side_chain_for_final" ligand_areas="inhibitor_final_sc"/>
			<InterfaceBuilder name="backbone" ligand_areas="inhibitor_final_bb" extension_window="3"/>
		</INTERFACE_BUILDERS>

		<MOVEMAP_BUILDERS>
			<MoveMapBuilder name="docking" sc_interface="side_chain_for_docking" minimize_water="false"/>
			<MoveMapBuilder name="final" sc_interface="side_chain_for_final" bb_interface="backbone" minimize_water="false"/>
		</MOVEMAP_BUILDERS>

		<SCORINGGRIDS ligand_chain="A" width="15">
			<ClassicGrid grid_name="classic" weight="1.0"/>
		</SCORINGGRIDS>

		<MOVERS>
			<Transform name="transform" chain="A" box_size="7.0" move_distance="0.2" angle="20" cycles="30" repeats="1" temperature="5"/>
			<HighResDocker name="high_res_docker" cycles="6" repack_every_Nth="3" scorefxn="ligand_soft_rep" movemap_builder="docking"/>
			<FinalMinimizer name="final" scorefxn="hard_rep" movemap_builder="final"/>
			<InterfaceScoreCalculator name="add_scores" chains="A" scorefxn="hard_rep" native= '/content/drive/MyDrive/nb_caffeine_mod_for_AFL/Inputs/aftNb_start.pdb'/>
		</MOVERS>

		<PROTOCOLS>
			<Add mover_name="transform"/>
			<Add mover_name="high_res_docker"/>
			<Add mover_name="final"/>
			<Add mover_name="add_scores"/>
		</PROTOCOLS>


</ROSETTASCRIPTS>
""").get_mover("ParsedProtocol")

protocols.rosetta_scripts.RosettaScriptsParser: Validating input script...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Parsed script:
<ROSETTASCRIPTS>
	<SCOREFXNS>
		<ScoreFunction name="ligand_soft_rep" weights="ligand_soft_rep"/>
		<ScoreFunction name="hard_rep" weights="ligand"/>
	</SCOREFXNS>
	<LIGAND_AREAS>
		<LigandArea add_nbr_radius="true" all_atom_mode="false" chain="A" cutoff="6.0" name="inhibitor_dock_sc"/>
		<LigandArea add_nbr_radius="true" all_atom_mode="false" chain="A" cutoff="6.0" name="inhibitor_final_sc"/>
		<LigandArea Calpha_restraints="0.3" add_nbr_radius="false" all_atom_mode="true" chain="A" cutoff="7.0" name="inhibitor_final_bb"/>
	</LIGAND_AREAS>
	<INTERFACE_BUILDERS>
		<InterfaceBuilder ligand_areas="inhibitor_dock_sc" name="side_chain_for_docking"/>
		<InterfaceBuilder ligand_areas="inhibitor_final_sc" name="side_chain_for_final"/>
		<InterfaceBuilder extension_window="3" ligand_areas="inhibitor_fin

In [29]:
scorefxn = pyrosetta.create_score_function("ligand_soft_rep")
if not os.getenv("DEBUG"):
    working_dir = os.getcwd()
    output_dir = "output"
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    os.chdir(output_dir)

    jd = pyrosetta.toolbox.py_jobdistributor.PyJobDistributor(pdb_name="aftNb_start.pdb", nstruct=5, scorefxn=scorefxn) # Output name, number of structures, scorefunction used
    jd.native_pose = pose
    #df = pd.DataFrame()
    while not jd.job_complete:
        test_pose = pose.clone()
        xml.apply(test_pose)
        jd.output_decoy(test_pose)
    os.chdir(working_dir)

Working on decoy: aftNb_start.pdb_4.pdb
protocols.rosetta_scripts.ParsedProtocol: =======================BEGIN MOVER Transform - transform=======================
protocols.rosetta_scripts.ParsedProtocol: [ ERROR ] Exception while processing protocol: 

File: /home/benchmark/rosetta/source/src/core/pose/chains_util.cc:455
chain_id A represents more than one chain!


RuntimeError: 

File: /home/benchmark/rosetta/source/src/core/pose/chains_util.cc:455
chain_id A represents more than one chain!